## The First Way

#### Wednesday 07/01/2020

The first step for making multiple sequence alignments (MSAs) for use with PAML to calculate dN/dS ratios is to identify the sequences that you would like to align. So, for each horizontally transferred gene, I need to select a number of homologous proteins for which I will estimate dN/dS ratios. The way I've done this is by BLASTing all of the HGT proteins against the nr database and taking the top 25 hits for each one, finding their CDS, and then using MACSE to align the CDS. I did this using small scripts I wrote that I have pasted below:

In [ ]:
#!/usr/bin/env bash
#blast.sh

#gunzip makeblastdb -in all_wolbachia_genomes.fna -out wolbachia_db -title wolbachia_db -dbtype nucl

find . -name "prot.part*.faa" | parallel -j 4 "blastp -query {} -out {}.out -db /Scratch/smnieves/databases/nr/nr -num_threads 8 -outfmt 6 -max_target_seqs 25"

This is followed by ProteinGrouper.py to group the HGT proteins with all of their hits. Then this small script to retrieve all the CDS for each protein accession (I tested this and it should work, I'm **very** excited about it):

In [ ]:
FilterBlast.py -dir ./ -percID 80 -len_align 100
cat *_nolimit.out.filtered > hgtvnr_nolimit.out.filtered
ProteinGrouper.py -file hgtvnr_nolimit.out.filtered

In [ ]:
# for creating accession lists for each homologous protein set
for file in *.filtered
do
awk '{print $1}' $file | sort -u > ${file%.faa.out.filtered}_filtered.acc
awk '{print $2}' $file | sort -u >> ${file%.faa.out.filtered}_filtered.acc
done

# for fetching fastas using protein blast accessions
for file in *.acc
do
while read -r line
do
efetch -db protein -format fasta_cds_na -id $line >> ${file%.acc}.fna
done < $file
done

# for fetching fastas using nucleotide blast accessions
for file in protein_aln*.txt
do
outfile=${file%.txt}.fna
faSomeRecords prot_cds.fna $file $outfile
blastdbcmd -entry_batch $file -db /Scratch/smnieves/databases/nt/nt >> $outfile
done

## The Better Way

#### Monday 08/31/2020 - Doing it over a new way
So I probably made several mistakes above by making something more complicated than it needs to be. The *new* plan is the following:
- [x] Download fastas for each protein
- [x] Make a separate file for each fasta
- [x] BLAST each fasta independently at nr
- [ ] Retrieve fastas for all hits to nr
- [ ] Make alignments
- [ ] Trim alignments
- [ ] Make trees
- [ ] PAML (profit!)

Below I outlined the method for obtaining the fastas for each protein, because NCBI Batch Entrez online is super unreliable:

In [ ]:
while read -r line
do
 efetch -db protein -format fasta -id $line >> all_prot.faa 
done < ~/bin/arth_queries.txt

Next, I make accession lists for all of the BLAST hits. After this I extract those accessions from the nr database.

In [1]:
# filter for hits greater than 100 aa alignments
FilterBlast.py -len_align 100 -percID 70 -dir ./

# create accession lists
for file in *.filtered
do
awk '{print $1}' $file | sort -u > ${file%.faa.out.filtered}.acc
awk '{print $2}' $file | sort -u >> ${file%.faa.out.filtered}.acc
done

# extract fastas
for file in *.acc
do
echo $file
outfile=${file%.acc}.fasta
epost -db protein -input $file | elink -target nuccore -db protein -name protein_nuccore | efetch -format fasta_cds_na | seqkit seq -w 0 | grep -A 1 -Ff $file > $outfile
sleep 1
done

# redo extraction on files that didn't work
for file in $(find . -name "*[0-9].fasta" -size 0)
do
echo ${file%.fasta}.acc
outfile=$file
epost -db protein -input ${file%.fasta}.acc | elink -target nuccore -db protein -name protein_nuccore | efetch -format fasta_cds_na | seqkit seq -w 0 | grep -A 1 -Ff ${file%.fasta}.acc > $outfile
sleep 3
done


SyntaxError: invalid syntax (<ipython-input-1-8f9c27c75b98>, line 2)

It may be useful for sanity checks to make sure the resulting multi-fasta file has the same number of sequences as accessions in the acc file. To do this, you can do the following:

In [ ]:
for file in *.acc
do
echo ${file%.acc}
cat $file | wc -l
cat ${file%.acc}.fasta | grep -c '^>'
sleep 3
done

#### Wednesday 09/03/2020 - Correcting a potentially horrendous mistake
So I realized that there was a problem with my **FilterBlast.py** program where it only kept hits with very low evalues. This was because I set my default evalue to 0, which is problematic for all the tests I did in the past.

The first thing I'm going to do is go back and look at my *initial BLAST against my Wolbachia database*. I want to see if I excluded any proteins that I now need to do tests for. **Luckily it doesn't seem like I used FilterBlast.py in any other step.**

#### Wednesday 09/16/2020 - FINALLY Downloading CDS for **WP_** Proteins
I just edited *The Better Way* above. It now **actually** is able to find the CDS for non-redundant protein accessions. What I have to do is fetch every fasta on the contig containing the WP accession, linearize them, and then grep for the sequence i want. It is a huge pain in the ass and pretty slow, but for now it works and I have not found another way.

Below is me trying to implement the above in python for better error handling, but I shelved it. I realized I can actually do the whole process once for each file, instead of once per line per file, which should produce fewer errors. I'm editing the above code to reflect this.

In [ ]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

__author__ = 'Serafina Nieves'
__email__ = 'smnieves@ucsc.edu'

from Bio import Entrez

parser = argparse.ArgumentParser(
            description='Handle filtering arguments for the FilterBlast class.',
            prefix_chars='-',
            usage='prog [options] -option1[default] <input >output'
        )

parser.add_argument('-dir', action='store',
                                 help='specify a valid path to directory containing protein accession files')
parser.add_argument('-ext', action='store', default='.acc',
                                 help='specify file type extension to correctly locate files')

args = self.parser.parse_args()

Entrez.email = "smnieves@ucsc.edu"

def get_proteins(file, outfile):
    Entrez.epost(file)
        
    



directory_in_str = parser.args.dir
    directory = os.fsencode(directory_in_str)

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        outfile = filename[:-4] + ".fasta"
        if filename.endswith(parser.args.ext):
            fastas = get_proteins(directory_in_str + filename, directory_in_str + outfile)

#### Thursday 09/24/2020 - Finalizing CDS Retrieval
The method outlined above for retrieving CDS seems to be working. It does take multiple passes to ensure that every query worked and downloaded a CDS. The good thing is that I think the whole query batch will fail, not just one accession out of the batch. This means I can search for files of 0 byte size to find the accession lists that need to be retried, instead of searching to see if individual accesssions downloaded.

## Creating Alignments

#### Tuesday 09/22/2020 - Generating alignments
The Entrez Direct process for downloading CDS takes a lot of time, so I am going to start generating the alignment files. The way I'll do this is by selecting the fasta files that are larger than 0 bytes, and then aligning those with MACSE. I will ask Russ tomorrow if I need to remove 100% identical sequences from these files.

In [ ]:
find . -name "*.fasta" -size +0 | parallel -j 8 "java -jar ~/bin/macse_v2.03.jar -prog alignSequences -seq {}"

#### Thursday 09/24/2020 - Trimming Alignments
MACSE seems to work fairly well for the coding sequence multi-fasta inputs. However, some alignments show large gaps both on the ends and in the middles of sequences. **TrimAl** is an awesome tool to deal with this, as it appears I can trip gaps off the ends and in the middles. Most importantly though, I can specify that I don't want to keep gaps that appear in more than 30% of sequences. In other words, I can specify that there must be a nucleotide present at that position in at least 70% of sequences or it needs to go. I can additionally specify that I don't want to trim this gap if it means less than 60% of the original alignment will be kept in the new alignment. This seems like a super useful feature.

In [ ]:
find . -name "*NT.fasta" | parallel -j 100 "trimal -gt .85 -cons 60 -in {} -out {}.trimal" 

Questions to ask Russ:
- [ ] Do I just need to run ModelFinder once and then use the same model to generate every tree
- [ ] For PAML should I use the branch model, site model, or branch-site model (I'm leaning towards branch-site)
- [ ] Tell him that I put a high threshold for the number of sequences that had to have a nucleotide at a certain position because PAML deals poorly with gaps
- [ ] How do I parse the PAML output files (programmatically or with my eyes lol)
- [ ] What model (how many dN/dS ratios for codons) in the control file?

#### Friday 09/25/2020 - Creating workable PHYLIP files
TrimAl only supports phylip and phylip3.2. The problem is that PAML does not like the phylip3.2 format, but the original phylip format from TrimAl does not allow you to have longer sequence identifiers. This is problematic because it creates duplicate sequence identifiers in the alignment due to truncation. Below I will use Biopython to write my alignment files out to relaxed PHYLIP format. *Additionally this can create the control file for PAML.*

In [8]:
from Bio import AlignIO

alignment_fasta = AlignIO.read("all_prot.part-1156_NT.trimal", "fasta")

for record in alignment_fasta:
    record.id = record.id[0:45].replace("|", "_") + "  "
    #print(record.id)

unique_seq = set()
for record in alignment_fasta:
    if record.seq in unique_seq and record.id.contains("WP_"):
        alignment_fasta.pop(record.index())
        
with open("all_prot.part-1156_NT.phy", "w") as outfile:
    AlignIO.PhylipIO.SequentialPhylipWriter(outfile).write_alignment(unique_seq, id_width=47)
    #AlignIO.write(alignment_fasta, outfile, "phylip-relaxed")
    

In [8]:
import sys

# create control files for PAML

file_base = str(sys.argv[1])
model = str(sys.argv[2])

with open("/home/smnieves/bin/codeml.ctl", "r") as ctl_template:
    template = ctl_template.readlines()
    
template[0] = "      seqfile = {0} * sequence data filename\n".format(file_base + ".phy")
template[1] = "     treefile = {0}      * tree structure file name\n".format(file_base + ".phy.treefile")
template[18] = "       model = {0}\n".format(model)

with open("codeml.ctl", 'w') as file:
    file.writelines(template)

<_io.TextIOWrapper name='all_prot.part-1156_NT.phy' mode='w' encoding='cp1252'>


In [17]:
from Bio.Phylo.PAML import codeml

wdir = str(sys.argv[1])
seqfile = str(sys.argv[2])
treefile = str(sys.argv[3])
mod = str(sys.argv[4])

cml = codeml.Codeml(working_dir=wdir, alignment=seqfile, tree=treefile, out_file="paml_results.out")

cml.set_options(noisy=9, verbose=1, runmode=0, seqtype=1, CodonFreq=2,
               ndata=10, clock=0, aaDist=0, model=mod, NSsites=[0], icode=0,
               Mgene=0, fix_kappa=0, kappa=2, fix_omega=0, omega=1,
               fix_alpha=1, alpha=0., Malpha=0, ncatG=8, getSE=0, RateAncestor=1,
               Small_Diff=.5e-6, cleandata=1, fix_blength=1, method=0)
cml.print_options()
cml.run()


NameError: name 'mod' is not defined


## Overview of AlntoPAML Pipeline

#### Friday 10/23/2020 - Snakefiles
So I decided instead of trying to string together a bunch of independent programs in a bash script, I was going to figure out snakemake. And I did! This allows me to:
- Stop all jobs and restart them where they left off
- Rerun the pipeline from any point
- Rerun the pipeline and regenerate downstream files
- Rerun the pipeline to generate missing dependencies
- Keep all of my rules organized
- Keep track of versions so I don't overwrite files by mistake
- Easily assign resources so I don't have to manage them for each step in the pipeline


So far this has worked really well. Below is the snakefile I used for the pipeline, named **AlntoPAML.snakefile**. Following it are any custom programs written for the purpose of this project. Programs that I did not write that are available on the command line do not appear below.

In [ ]:
IDS = range(1, 1673)

rule all:
        input:
                #expand("prot_{file}/paml_results_null.out", file=IDS),
                #expand("prot_{file}/paml_results_alt.out", file=IDS),
                expand("prot_{file}/CodemlModelCompare_prot_{file}.txt", file=IDS)

rule macse_alignseqs:
        input:
                "all_prot.part-{file}.fasta"
        output:
                NT = "prot_{file}/all_prot.part-{file}_aligned_NT.fasta",
                AA = "prot_{file}/all_prot.part-{file}_aligned_AA.fasta"
        shell:
                "java -jar ~/bin/macse_v2.03.jar -prog alignSequences -seq {input} "
                "-out_NT {output.NT} -out_AA {output.AA}"

rule trim_alignments:
        input:
                "prot_{file}/all_prot.part-{file}_aligned_NT.fasta"
        output:
                "prot_{file}/all_prot.part-{file}.trimal"
        shell:
                "trimal -gt .85 -resoverlap .75 -seqoverlap .85 -in {input} -out {output}"

rule macse_realignseqs:
        input:
                "prot_{file}/all_prot.part-{file}.trimal"
        output:
                NT = "prot_{file}/all_prot.part-{file}_refined_NT.fasta",
                AA = "prot_{file}/all_prot.part-{file}_refigned_AA.fasta"
        shell:
                "java -jar ~/bin/macse_v2.03.jar -prog refineAlignment -align {input} "
                "-out_NT {output.NT} -out_AA {output.AA}"

rule macse_exportaln:
        input:
                "prot_{file}/all_prot.part-{file}_refined_NT.fasta"
        output:
                NT = "prot_{file}/all_prot.part-{file}_noFSstop_NT.fasta",
                AA = "prot_{file}/all_prot.part-{file}_noFSstop_AA.fasta"
        shell:
                "java -jar ~/bin/macse_v2.03.jar -prog exportAlignment -align {input} -codonForInternalStop NNN "
                "-codonForFinalStop --- -codonForInternalFS --- -charForRemainingFS - "
                "-out_NT {output.NT} -out_AA {output.AA}"

rule generate_phylip:
        input:
                "prot_{file}/all_prot.part-{file}_noFSstop_NT.fasta"
        output:
                "prot_{file}/all_prot.part-{file}.phy"
        shell:
                "/home/smnieves/bin/WritePhylip.py {input} {output}"

rule generate_tree:
        input:
                "prot_{file}/all_prot.part-{file}.phy"
        output:
                "prot_{file}/all_prot.part-{file}.phy.treefile"
        shell:
                "iqtree -s {input} -m GTR+G -redo -quiet"

rule annotate_tree:
        input:
                "prot_{file}/all_prot.part-{file}.phy.treefile"
        output:
                "prot_{file}/all_prot.part-{file}.phy.foreground.treefile"
        shell:
                "AnnotateTree.py {input} {output}"

rule run_paml_null:
        input:
                seqfile="prot_{file}/all_prot.part-{file}.phy",
                treefile="prot_{file}/all_prot.part-{file}.phy.treefile"
        params:
                wdir="prot_{file}",
                model=0
        output:
                "prot_{file}/paml_results_null.out"
        shell:
                "PAML.py {params.wdir} {input.seqfile} {input.treefile} {params.model} {output}"

rule run_paml_alt:
        input:
                seqfile="prot_{file}/all_prot.part-{file}.phy",
                treefile="prot_{file}/all_prot.part-{file}.phy.foreground.treefile"
        params:
                wdir="prot_{file}",
                model=2
        output:
                "prot_{file}/paml_results_alt.out"
        shell:
                "PAML.py {params.wdir} {input.seqfile} {input.treefile} {params.model} {output}"

rule codeml_compare:
        input:
                null="prot_{file}/paml_results_null.out",
                alt="prot_{file}/paml_results_alt.out"
        output:
                output="prot_{file}/CodemlModelCompare_prot_{file}.txt"
        params:
                wdir="prot_{file}"
        shell:
                "CodemlCompare.py {params.wdir} {input.null} {input.alt} {output}"

In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

__author__ = 'Serafina Nieves'
__email__ = 'smnieves@ucsc.edu'

'''WritePhylip.py writes a sequential Phylip file without sequence duplicates, limiting the sequence alignment to at most 50 sequences.'''

from Bio import AlignIO
from Bio import Align
import random
import sys


alignment_fasta = AlignIO.read(str(sys.argv[1]), "fasta")

clean_alignment = Align.MultipleSeqAlignment([])

arth_list = []
with open("/home/smnieves/bin/arth_queries.txt") as prot_list:
    for line in prot_list:
        arth_list.append(line.strip())

for record in alignment_fasta:
    record.id = record.id[0:45].replace("|", "_") + "  "

def deduplicate(alignment):
    dedup_alignment = Align.MultipleSeqAlignment([])
    unique_prot = set()
    for record in alignment:
        prot_name = record.id[record.id.index("cds_") + 4: record.id.index(".", record.id.index("cds_")) + 2]
        if prot_name not in unique_prot:
            unique_prot.add(prot_name)
            dedup_alignment.append(record)
    return(dedup_alignment)

clean_alignment = deduplicate(alignment_fasta)

if len(clean_alignment) > 50:
    clean_alignment = random.sample(list(clean_alignment), 50)

for record in alignment_fasta:
    if any(acc in record.id for acc in arth_list):
        clean_alignment.append(record)

output_alignment = deduplicate(clean_alignment)


with open(sys.argv[2], "w") as outfile:
    AlignIO.PhylipIO.SequentialPhylipWriter(outfile).write_alignment(output_alignment, id_width=50)
    #AlignIO.write(alignment_fasta, outfile, "phylip-relaxed")

FileNotFoundError: [Errno 2] No such file or directory: '-f'

In [ ]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

__author__ = 'Serafina Nieves'
__email__ = 'smnieves@ucsc.edu'

import sys
from Bio import Phylo

'''AnnotateTree.py is a program that takes a list of accessions and annotates branches on the tree with those accessions as foreground branches.'''

arth_list = []
with open("/home/smnieves/bin/arth_queries.txt") as prot_list:
    for line in prot_list:
        arth_list.append(line.strip())

treefile = str(sys.argv[1])
outfile = str(sys.argv[2])

newick = Phylo.read(treefile, "newick")
for clade in newick.find_clades(name=True):
    for acc in arth_list:
        try:
            if acc in clade.name:
                clade.name = clade.name + "#1"
        except(TypeError):
             continue
# for clade in newick.find_clades():
    # print(clade.name)

Phylo.write(newick, outfile, format="newick")

In [ ]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

__author__ = 'Serafina Nieves'
__email__ = 'smnieves@ucsc.edu'

from Bio.Phylo.PAML import codeml
import sys

''' PAML.py is a program that allows fast customization of PAML parameters and reproducible control files. It can specify the parameters and
the working directory to keep files together. The parameters can be written to a control file and the output file can be parsed (not currently implemented).'''

wdir = str(sys.argv[1])
seqfile = str(sys.argv[2])
treefile = str(sys.argv[3])
mod = str(sys.argv[4])
outfile= str(sys.argv[5])

cml = codeml.Codeml(working_dir=wdir, alignment=seqfile, tree=treefile, out_file=outfile)

cml.set_options(noisy=9, verbose=1, runmode=0, seqtype=1, CodonFreq=2,
               ndata=0, clock=0, aaDist=0, model=mod, NSsites=[0], icode=0,
               Mgene=0, fix_kappa=0, kappa=2, fix_omega=0, omega=1,
               fix_alpha=1, alpha=0., Malpha=0, ncatG=8, getSE=0, RateAncestor=0,
               Small_Diff=.5e-6, cleandata=1, fix_blength=1, method=0)

cml.print_options()
cml.run(verbose=True)

In [ ]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

__author__ = 'Serafina Nieves'
__email__ = 'smnieves@ucsc.edu'

from Bio.Phylo.PAML import codeml
import sys
import numpy as np
from scipy.stats.distributions import chi2
import matplotlib.pyplot as plt

''' CodemlCompare.py compares the null and 2 ratio model of codeml to determine which model fits the data better. '''

# set parameters for the program (flags)
x_points = []
y_points = []
string_towrite = []


num_list = range(1, 1673)
for num in num_list:
    try:
        wdir = "prot_" + str(num)
        infile_null = wdir + "/paml_results_null.out"
        infile_alt = wdir + "/paml_results_alt.out"
        outfile = "/Scratch/smnieves/alignments/CodemlModelCompare.txt"
        outfile2 = wdir + "/CodemlCompare_prot" + str(num) + ".txt"

        # read in data and parse it for relevant values
        print(infile_null)
        results_null = codeml.read(infile_null)
        results_alt = codeml.read(infile_alt)

        lnL_null = results_null.get("NSsites").get(0).get("lnL")
        lnL_alt = results_alt.get("NSsites").get(0).get("lnL")

        likelihood_ratio = -2 * (lnL_null - lnL_alt)
        p_value = chi2.sf(likelihood_ratio, 1)

        arth_omega = results_alt.get("NSsites").get(0).get("parameters").get("omega")[1]
        background_omega = results_alt.get("NSsites").get(0).get("parameters").get("omega")[0]

        # save point data and plot points
        print(background_omega, arth_omega)
        x_points.append(background_omega)
        y_points.append(arth_omega)

        # create formatted strings
        genes_interest = []
        arth_list = open("/home/smnieves/bin/arthqueries.txt").readlines()
        if p_value <= 0.05:
            dir_num = wdir[5:]
            acc_file = "all_prot.part-" + dir_num + ".acc"
            for line in open(acc_file):
                if line in arth_list:
                    genes_interest.append(line.strip())

        string_towrite.append("directory: {0} \t likelihood ratio: {1} \t p-value: {2} \t arth_genes: {3} \n".format(wdir, likelihood_ratio, p_value, ",".join(genes_interest)))

    except FileNotFoundError:
        continue
        
# change dimensions of the entire figure
figureHeight = 5
figureWidth = 5

plt.figure(figsize=(figureWidth, figureHeight))

mainPanelWidth = 4
mainPanelHeight = 4

mainPanel = plt.axes([0.1, 0.1, mainPanelWidth / figureWidth, mainPanelHeight / figureHeight])

mainPanel.scatter(x_points, y_points)
plt.savefig('test_fig.png', dpi=300)

# write formatted strings to files
with open(outfile, 'w') as file:
    for string in string_towrite:
        file.write(string)

with open(outfile2, 'w') as file2:
    for string in string_towrite:
        file2.write(string)

## Characteristics of Transferred Genes

#### Saturday 02/20/2021 - Insertion Locations
This is the code I used to do a BLAST to find the insertion locations of the transferred genes. I added qlen and slen so that I could find the subject coverage of the alignment, keeping only subject coverages less than 10% of an arthropod genes length.

In [ ]:
find . -name "*.faa" | parallel -j 8 "blastp -query {} -out {}.out -taxidlist arthropoda_taxids.txt -db /Scratch/smnieves/databases/nr/nr -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen staxids sscinames scomnames sskingdoms' -num_threads 4"

These BLAST out files are then filtered with awk, looking for alignments of the query and subject that constitute less than 10% of the subject's overall length, and that have 80% sequence identity in the aligned region. These files are named with the scheme: file.faa.out_10aln_80id.filtered, where the 10aln specifies that less than 10% of the subject sequence must be aligned and that greater than 80% sequence identity must be present.

In [ ]:
for file in *.out; do cat $file | awk '$4/$13 < .1 && $3 > 90' > ${file}_10aln_90id.filtered; done
find . -name '*.filtered' -empty -type f -delete
ls *filtered